In [ ]:
from pymongo import MongoClient
from collections import deque
from metadataclient.api import (insert_run_start, insert_descriptor,
                                insert_run_stop, insert_event, bulk_insert_events)
from metadataclient.api import (find_run_starts, find_descriptors, find_events,
                               find_run_stops, find_last)
from metadataclient.conf import connection_config
from urllib3.exceptions import ConnectionError


In [ ]:
class MdsMigrationException(RuntimeError):
    """Goodbye cruel world!"""
    pass

# Before this script is run, make sure you create a database called metadataservice

# TODO: Switch to deque from OrderedDict if too slow

**_Ensure global connection management is functional by providing fake host information_**

In [ ]:
try:
    MONGO_HOST = 'kusadasi'
    connection_config['host'] = MONGO_HOST 
    res = next(find_last())
except Exception:
    pass

**Connection Management**

In [ ]:
MONGO_HOST = 'localhost'
MONGO_PORT = 27017
MIGRATION_DB = 'datastore2'
# assuming both mongo and tornado server on the same machine
connection_config['host'] = MONGO_HOST 
connection_config['port'] = 7770
connection_config['database'] = 'metadataservice'
pymongo_client = MongoClient(MONGO_HOST, MONGO_PORT)
database = pymongo_client[MIGRATION_DB]
try:
    res = (next(find_last()))
except Exception:
    raise MdsMigrationException('Seems like metadataservice is not accessible or empty')

**Get all run_starts**

In [ ]:
print(database) # visual sanity check


In [ ]:
def rstart_oid_lookup(database, oid):
    try:
        res = next(database.run_start.find({'_id':oid}))
    except StopIteration:
        raise MdsMigrationException('Seems like run_start with this oid does not exist')
    return res

In [ ]:
def descriptor_oid_lookup(database, oid):
    try:
        res = next(database.event_descriptor.find({'_id':oid}))
    except StopIteration:
        raise MdsMigrationException('Seems like descriptor with this oid does not exist')
    return res

**Save all run_starts into an OrderedDict()**

In [ ]:
rstart_crsr = database.run_start.find()
run_starts = deque()
for rstart in rstart_crsr:
    run_starts.append(rstart)

**Go over all beamline configurations, delete run_start reference fields, add as a embedded document to run_start, and update entry in run_starts deque()**

In [ ]:
bcfg_crsr = database.beamline_config.find()
for cfg in bcfg_crsr:
    runstart = rstart_oid_lookup(database, cfg['run_start']) 
    # check if run_start really exists!
    del(cfg['run_start'])
    runstart['config'] = cfg
    run_starts.append(rstart)
print (run_starts)

**Get all descriptors and insert them using metadataclient.Iterate over descriptors, replace run_start oid references with run_start uid foreign keys, and save them to a deque and a dict that is keyed on uids(for caching)**

In [ ]:
desc_crsr = database.event_descriptor.find()
descriptors = deque()
desc_dict = {}
for desc in desc_crsr:
    rstart = rstart_oid_lookup(database, desc['run_start'])
    # make sure run_start exists
    desc['run_start'] = run_starts[rstart['uid']]['uid'] 
    desc_dict[desc['uid']] = desc
    descriptors.append(desc)

**Insert all run_starts and descriptors so far**

In [ ]:
# insert via metadataclient

In [ ]:
for d in descriptors:
    events = deque()
    event_crsr = database.event_descriptor.find({'descriptor': d['_id']})
    for e in event_crsr:
        e['descriptor'] = d['uid']
        events.append(e)
    # bulk inserts events via metadataclient